# 6. Recent Advances in Deep Learning

## 6.1. Important Remarks

If you are accessing this tutorial via [Google Colab](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/training.ipynb), first make sure to use Google Colab in English. This will help us to better assist you with issues that might arise during the tutorial. There are two ways to change the default language if it isn't English already:
1. On Google Colab, go to `Help` -> `View in English` 
2. Change the default language of your browser to `English`.

To also ease the communication when communicating errors, enable line numbers within the settings of Colab.

1. On Google Colab, go to `Tools` -> `Settings` -> `Editor` -> `Show line numbers`

In general, we strongly advise you to use Google Colab as it provides you with a working Python distribution as well as free GPU resources. To make Colab use GPUs, you need to change the current notebooks runtime type via:

- `Runtime` -> `Change runtime type` -> `Dropdown` -> `GPU` -> `Save`

**Hint:** you can auto-complete code in Colab via `ctrl` + `spacebar`

For the live tutorial, we require all participants to use Colab. If you decide to rerun the tutorial at later points and rather want to have it run locally on your machine, feel free to clone our [GitHub repository](https://github.com/mariusbock/dl-for-har).

To get started with this notebook, you need to first run the code cell below. Please set `use_colab` to be `True` if you are accessing this notebook via Colab. If not, please set it to `False`. This code cell will make sure that imports from our GitHub repository will work.

In [1]:
import os, sys

use_colab = True

module_path = os.path.abspath(os.path.join('..'))

if use_colab:
    # move to content directory and remove directory for a clean start 
    %cd /content/         
    %rm -rf dl-for-har
    # clone package repository (will throw error if already cloned)
    !git clone https://github.com/mariusbock/dl-for-har.git
    # navigate to dl-for-har directory
    %cd dl-for-har/       
else:
    os.chdir(module_path)
    
# this statement is needed so that we can use the methods of the DL-ARC pipeline
if module_path not in sys.path:
    sys.path.append(module_path)

/content
Cloning into 'dl-for-har'...
remote: Enumerating objects: 940, done.
remote: Counting objects: 100% (940/940), done.
remote: Compressing objects: 100% (624/624), done.
remote: Total 940 (delta 445), reused 789 (delta 306), pack-reused 0
Receiving objects: 100% (940/940), 25.65 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (445/445), done.
/content/dl-for-har


## 6.2. Loading and preparing the data

In [2]:
import pandas as pd
import numpy as np
import warnings

from data_processing.sliding_window import apply_sliding_window

warnings.filterwarnings('ignore')


# data loading
data_folder = 'data'
dataset = 'rwhar_3sbjs_data.csv'
data = pd.read_csv(os.path.join(data_folder, dataset), names=['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity_label'])
print("\nValue counts before label encoding: ")
print(data['activity_label'].value_counts())

# all activity names (you need them to define the label_dict!)
class_names = ['climbing_down', 'climbing_up', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking']

# label encoding dict
label_dict = {
    'climbing_down': 0,
    'climbing_up': 1,
    'jumping': 2,
    'lying': 3,
    'running': 4,
    'sitting': 5,
    'standing': 6,
    'walking': 7
}

# apply label encoding
data['activity_label'] = data['activity_label'].replace(label_dict) 

# define the train data to be all data belonging to the first two subjects
train_data = data[data.subject_id <= 1]
# define the validation data to be all data belonging to the third subject
valid_data = data[data.subject_id == 2]

# settings for the sliding window (change them if you want to!)
sw_length = 50
sw_unit = 'units'
sw_overlap = 50

# apply a sliding window on top of both the train and validation data; you can use our predefined method
X_train, y_train = apply_sliding_window(train_data.iloc[:, :-1], train_data.iloc[:, -1], sliding_window_size=sw_length, unit=sw_unit, sampling_rate=50, sliding_window_overlap=sw_overlap)
X_valid, y_valid = apply_sliding_window(valid_data.iloc[:, :-1], valid_data.iloc[:, -1], sliding_window_size=sw_length, unit=sw_unit, sampling_rate=50, sliding_window_overlap=sw_overlap)

# (optional) omit the first feature column (subject_identifier) from the train and validation dataset
X_train, X_valid = X_train[:, :, 1:], X_valid[:, :, 1:]

# convert the features of the train and validation to float32 and labels to uint8 for GPU compatibility 
X_train, y_train = X_train.astype(np.float32), y_train.astype(np.uint8)
X_valid, y_valid = X_valid.astype(np.float32), y_valid.astype(np.uint8)


Value counts before label encoding: 
running          99204
walking          96810
sitting          95265
standing         94106
lying            94038
climbing_up      87572
climbing_down    78004
jumping          14261
Name: activity_label, dtype: int64


## 6.3. The Label Smoothing Loss Function

In [3]:
from torch import nn


class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.0):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, prediction, target):
        assert 0 <= self.smoothing < 1
        neglog_softmaxPrediction = -prediction.log_softmax(dim=1)

        with torch.no_grad():
            smoothedLabels = self.smoothing / (prediction.size(1) - 1)* torch.ones_like(prediction)
            smoothedLabels.scatter_(1, target.data.unsqueeze(1), 1-self.smoothing)
        return torch.mean(torch.sum(smoothedLabels * neglog_softmaxPrediction, dim=1)) 

## 6.4. Training with and without Label Smoothing

### 6.4.1. Define the Config Object

In [5]:
config = {
    #### TRY AND CHANGE THESE PARAMETERS ####
    # sliding window settings
    'sw_length': 50,
    'sw_unit': 'units',
    'sampling_rate': 50,
    'sw_overlap': 30,
    # network settings
    'nb_conv_blocks': 2,
    'conv_block_type': 'normal',
    'nb_filters': 64,
    'filter_width': 11,
    'nb_units_lstm': 128,
    'nb_layers_lstm': 1,
    'drop_prob': 0.5,
    # training settings
    'epochs': 30,
    'batch_size': 100,
    'loss': 'cross_entropy',
    'use_weights': True,
    'weights_init': 'xavier_uniform',
    'optimizer': 'adam',
    'lr': 1e-4,
    'weight_decay': 1e-6,
    ### UP FROM HERE YOU SHOULD RATHER NOT CHANGE THESE ####
    'batch_norm': False,
    'dilation': 1,
    'pooling': False,
    'pool_type': 'max',
    'pool_kernel_width': 2,
    'reduce_layer': False,
    'reduce_layer_output': 10,
    'nb_classes': 8,
    'seed': 1,
    'gpu': 'cuda:0',
    'verbose': False,
    'print_freq': 10,
    'save_gradient_plot': False,
    'print_counts': False,
    'adj_lr': False,
    'adj_lr_patience': 5,
    'early_stopping': False,
    'es_patience': 5,
    'save_test_preds': False
}

### 6.4.3. With Label Smoothing

In [7]:
# initialize your DeepConvLSTM object 
network = DeepConvLSTM(config)

# sends network to the GPU and sets it to training mode
network.to(config['gpu'])
network.train()

# initialize the optimizer and loss
optimizer = torch.optim.Adam(network.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
criterion = LabelSmoothingLoss(0.1)

# define your training loop; iterates over the number of epochs
for e in range(config['epochs']):
    # helper objects needed for proper documentation
    train_losses = []
    train_preds = []
    train_gt = []
    start_time = time.time()
    batch_num = 1

    # initializes train dataset in Torch format
    dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    
    # define your trainloader; use from torch.utils.data import DataLoader
    trainloader = DataLoader(dataset, batch_size=config['batch_size'])

    # iterate over the trainloader object (it'll return batches which you can use)
    for i, (x, y) in enumerate(trainloader):
        # sends batch x and y to the GPU
        inputs, targets = x.to(config['gpu']), y.to(config['gpu'])
        optimizer.zero_grad()

        # send inputs through network to get predictions
        train_output = network(inputs)

        # calculates loss
        loss = criterion(train_output, targets.long())

        # backprogate your computed loss through the network
        # use the .backward() and .step() function on your loss and optimizer
        loss.backward()
        optimizer.step()

        # calculate actual predictions (i.e. softmax probabilites); use torch.nn.functional.softmax()
        train_output = torch.nn.functional.softmax(train_output, dim=1)

        # appends the computed batch loss to list
        train_losses.append(loss.item())

        # creates predictions and true labels; appends them to the final lists
        y_preds = np.argmax(train_output.cpu().detach().numpy(), axis=-1)
        y_true = targets.cpu().numpy().flatten()
        train_preds = np.concatenate((np.array(train_preds, int), np.array(y_preds, int)))
        train_gt = np.concatenate((np.array(train_gt, int), np.array(y_true, int)))

        # prints out every 100 batches information about the current loss and time per batch
        if batch_num % 100 == 0 and batch_num > 0:
            cur_loss = np.mean(train_losses)
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d} batches | ms/batch {:5.2f} | train loss {:5.2f}'.format(e, batch_num, elapsed * 1000 / config['batch_size'], cur_loss))
            start_time = time.time()
            batch_num += 1

    # helper objects
    val_preds = []
    val_gt = []
    val_losses = []

    # initialize validation dataset in Torch format
    dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_valid).float(), torch.from_numpy(y_valid))
    
    # define your valloader; use from torch.utils.data import DataLoader
    valloader = DataLoader(dataset, batch_size=config['batch_size'])

    # sets network to eval mode and 
    network.eval()
    with torch.no_grad():
        # iterate over the valloader object (it'll return batches which you can use)
        for i, (x, y) in enumerate(valloader):
            # sends batch x and y to the GPU
            inputs, targets = x.to(config['gpu']), y.to(config['gpu'])

            # send inputs through network to get predictions
            val_output = network(inputs)

            # calculates loss by passing criterion both predictions and true labels 
            val_loss = criterion(val_output, targets.long())

            # calculate actual predictions (i.e. softmax probabilites); use torch.nn.functional.softmax() on dim=1
            val_output = torch.nn.functional.softmax(val_output, dim=1)

            # appends validation loss to list
            val_losses.append(val_loss.item())

            # creates predictions and true labels; appends them to the final lists
            y_preds = np.argmax(val_output.cpu().numpy(), axis=-1)
            y_true = targets.cpu().numpy().flatten()
            val_preds = np.concatenate((np.array(val_preds, int), np.array(y_preds, int)))
            val_gt = np.concatenate((np.array(val_gt, int), np.array(y_true, int)))

        # print epoch evaluation results for train and validation dataset
        print("\nEPOCH: {}/{}".format(e + 1, config['epochs']),
                  "\nTrain Loss: {:.4f}".format(np.mean(train_losses)),
                  "Train Acc: {:.4f}".format(jaccard_score(train_gt, train_preds, average='macro')),
                  "Train Prec: {:.4f}".format(precision_score(train_gt, train_preds, average='macro')),
                  "Train Rcll: {:.4f}".format(recall_score(train_gt, train_preds, average='macro')),
                  "Train F1: {:.4f}".format(f1_score(train_gt, train_preds, average='macro')),
                  "\nVal Loss: {:.4f}".format(np.mean(val_losses)),
                  "Val Acc: {:.4f}".format(jaccard_score(val_gt, val_preds, average='macro')),
                  "Val Prec: {:.4f}".format(precision_score(val_gt, val_preds, average='macro')),
                  "Val Rcll: {:.4f}".format(recall_score(val_gt, val_preds, average='macro')),
                  "Val F1: {:.4f}".format(f1_score(val_gt, val_preds, average='macro')))

        # if chosen, print the value counts of the predicted labels for train and validation dataset
        if config['print_counts']:
            print('Predicted Train Labels: ')
            print(np.vstack((np.nonzero(np.bincount(train_preds))[0], np.bincount(train_preds)[np.nonzero(np.bincount(train_preds))[0]])).T)
            print('Predicted Val Labels: ')
            print(np.vstack((np.nonzero(np.bincount(val_preds))[0], np.bincount(val_preds)[np.nonzero(np.bincount(val_preds))[0]])).T)


    # set network to train mode again
    network.train()


EPOCH: 1/30 
Train Loss: 2.2150 Train Acc: 0.0550 Train Prec: 0.1202 Train Rcll: 0.1165 Train F1: 0.0970 
Val Loss: 1.9732 Val Acc: 0.1506 Val Prec: 0.1734 Val Rcll: 0.2499 Val F1: 0.2033

EPOCH: 2/30 
Train Loss: 2.0666 Train Acc: 0.0862 Train Prec: 0.1728 Train Rcll: 0.1396 Train F1: 0.1457 
Val Loss: 1.8832 Val Acc: 0.1721 Val Prec: 0.1912 Val Rcll: 0.3018 Val F1: 0.2109

EPOCH: 3/30 
Train Loss: 2.0197 Train Acc: 0.0980 Train Prec: 0.1724 Train Rcll: 0.1790 Train F1: 0.1653 
Val Loss: 1.9243 Val Acc: 0.1138 Val Prec: 0.1785 Val Rcll: 0.2647 Val F1: 0.1689

EPOCH: 4/30 
Train Loss: 1.8225 Train Acc: 0.2098 Train Prec: 0.3035 Train Rcll: 0.3000 Train F1: 0.2950 
Val Loss: 1.8468 Val Acc: 0.1258 Val Prec: 0.2963 Val Rcll: 0.2762 Val F1: 0.2001

EPOCH: 5/30 
Train Loss: 1.6994 Train Acc: 0.2657 Train Prec: 0.3504 Train Rcll: 0.3718 Train F1: 0.3544 
Val Loss: 1.7919 Val Acc: 0.1467 Val Prec: 0.2427 Val Rcll: 0.3156 Val F1: 0.2197

EPOCH: 6/30 
Train Loss: 1.6187 Train Acc: 0.3195 Trai

### 6.4.2. Without Label Smoothing

In [6]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

from model.DeepConvLSTM import DeepConvLSTM

import time

# define the missing parameters within the config file. 
# window_size = size of the sliding window in units
# nb_channels = number of feature channels
# nb_classes = number of classes that can be predicted
config['window_size'] = X_train.shape[1]
config['nb_channels'] = X_train.shape[2]
config['nb_classes'] = len(class_names)

# initialize your DeepConvLSTM object 
network = DeepConvLSTM(config)

# sends network to the GPU and sets it to training mode
network.to(config['gpu'])
network.train()

# initialize the optimizer and loss
optimizer = torch.optim.Adam(network.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
criterion = LabelSmoothingLoss()

# define your training loop; iterates over the number of epochs
for e in range(config['epochs']):
    # helper objects needed for proper documentation
    train_losses = []
    train_preds = []
    train_gt = []
    start_time = time.time()
    batch_num = 1

    # initializes train dataset in Torch format
    dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    
    # define your trainloader; use from torch.utils.data import DataLoader
    trainloader = DataLoader(dataset, batch_size=config['batch_size'])

    # iterate over the trainloader object (it'll return batches which you can use)
    for i, (x, y) in enumerate(trainloader):
        # sends batch x and y to the GPU
        inputs, targets = x.to(config['gpu']), y.to(config['gpu'])
        optimizer.zero_grad()

        # send inputs through network to get predictions
        train_output = network(inputs)

        # calculates loss
        loss = criterion(train_output, targets.long())

        # backprogate your computed loss through the network
        # use the .backward() and .step() function on your loss and optimizer
        loss.backward()
        optimizer.step()

        # calculate actual predictions (i.e. softmax probabilites); use torch.nn.functional.softmax()
        train_output = torch.nn.functional.softmax(train_output, dim=1)

        # appends the computed batch loss to list
        train_losses.append(loss.item())

        # creates predictions and true labels; appends them to the final lists
        y_preds = np.argmax(train_output.cpu().detach().numpy(), axis=-1)
        y_true = targets.cpu().numpy().flatten()
        train_preds = np.concatenate((np.array(train_preds, int), np.array(y_preds, int)))
        train_gt = np.concatenate((np.array(train_gt, int), np.array(y_true, int)))

        # prints out every 100 batches information about the current loss and time per batch
        if batch_num % 100 == 0 and batch_num > 0:
            cur_loss = np.mean(train_losses)
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d} batches | ms/batch {:5.2f} | train loss {:5.2f}'.format(e, batch_num, elapsed * 1000 / config['batch_size'], cur_loss))
            start_time = time.time()
            batch_num += 1

    # helper objects
    val_preds = []
    val_gt = []
    val_losses = []

    # initialize validation dataset in Torch format
    dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_valid).float(), torch.from_numpy(y_valid))
    
    # define your valloader; use from torch.utils.data import DataLoader
    valloader = DataLoader(dataset, batch_size=config['batch_size'])

    # sets network to eval mode and 
    network.eval()
    with torch.no_grad():
        # iterate over the valloader object (it'll return batches which you can use)
        for i, (x, y) in enumerate(valloader):
            # sends batch x and y to the GPU
            inputs, targets = x.to(config['gpu']), y.to(config['gpu'])

            # send inputs through network to get predictions
            val_output = network(inputs)

            # calculates loss by passing criterion both predictions and true labels 
            val_loss = criterion(val_output, targets.long())

            # calculate actual predictions (i.e. softmax probabilites); use torch.nn.functional.softmax() on dim=1
            val_output = torch.nn.functional.softmax(val_output, dim=1)

            # appends validation loss to list
            val_losses.append(val_loss.item())

            # creates predictions and true labels; appends them to the final lists
            y_preds = np.argmax(val_output.cpu().numpy(), axis=-1)
            y_true = targets.cpu().numpy().flatten()
            val_preds = np.concatenate((np.array(val_preds, int), np.array(y_preds, int)))
            val_gt = np.concatenate((np.array(val_gt, int), np.array(y_true, int)))

        # print epoch evaluation results for train and validation dataset
        print("\nEPOCH: {}/{}".format(e + 1, config['epochs']),
                  "\nTrain Loss: {:.4f}".format(np.mean(train_losses)),
                  "Train Acc: {:.4f}".format(jaccard_score(train_gt, train_preds, average='macro')),
                  "Train Prec: {:.4f}".format(precision_score(train_gt, train_preds, average='macro')),
                  "Train Rcll: {:.4f}".format(recall_score(train_gt, train_preds, average='macro')),
                  "Train F1: {:.4f}".format(f1_score(train_gt, train_preds, average='macro')),
                  "\nVal Loss: {:.4f}".format(np.mean(val_losses)),
                  "Val Acc: {:.4f}".format(jaccard_score(val_gt, val_preds, average='macro')),
                  "Val Prec: {:.4f}".format(precision_score(val_gt, val_preds, average='macro')),
                  "Val Rcll: {:.4f}".format(recall_score(val_gt, val_preds, average='macro')),
                  "Val F1: {:.4f}".format(f1_score(val_gt, val_preds, average='macro')))

        # if chosen, print the value counts of the predicted labels for train and validation dataset
        if config['print_counts']:
            print('Predicted Train Labels: ')
            print(np.vstack((np.nonzero(np.bincount(train_preds))[0], np.bincount(train_preds)[np.nonzero(np.bincount(train_preds))[0]])).T)
            print('Predicted Val Labels: ')
            print(np.vstack((np.nonzero(np.bincount(val_preds))[0], np.bincount(val_preds)[np.nonzero(np.bincount(val_preds))[0]])).T)


    # set network to train mode again
    network.train()


EPOCH: 1/30 
Train Loss: 2.1729 Train Acc: 0.0596 Train Prec: 0.1057 Train Rcll: 0.1288 Train F1: 0.1032 
Val Loss: 2.0512 Val Acc: 0.0701 Val Prec: 0.2099 Val Rcll: 0.1298 Val F1: 0.1184

EPOCH: 2/30 
Train Loss: 2.1090 Train Acc: 0.0480 Train Prec: 0.1173 Train Rcll: 0.1062 Train F1: 0.0878 
Val Loss: 1.9685 Val Acc: 0.0467 Val Prec: 0.0487 Val Rcll: 0.1521 Val F1: 0.0738

EPOCH: 3/30 
Train Loss: 1.9939 Train Acc: 0.0770 Train Prec: 0.1589 Train Rcll: 0.1713 Train F1: 0.1339 
Val Loss: 1.9444 Val Acc: 0.0325 Val Prec: 0.0375 Val Rcll: 0.1445 Val F1: 0.0565

EPOCH: 4/30 
Train Loss: 1.8732 Train Acc: 0.1172 Train Prec: 0.2097 Train Rcll: 0.2296 Train F1: 0.1911 
Val Loss: 1.9474 Val Acc: 0.0182 Val Prec: 0.0252 Val Rcll: 0.1237 Val F1: 0.0322

EPOCH: 5/30 
Train Loss: 1.7912 Train Acc: 0.1505 Train Prec: 0.2443 Train Rcll: 0.2738 Train F1: 0.2334 
Val Loss: 1.8003 Val Acc: 0.0728 Val Prec: 0.1523 Val Rcll: 0.2128 Val F1: 0.1184

EPOCH: 6/30 
Train Loss: 1.6953 Train Acc: 0.1961 Trai